In [0]:
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow import keras
import pandas as pd

In [0]:
df = pd.read_csv("iris.csv")
X = df.drop(["class"],axis=1)
Y = df['class'].map({'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2})

In [0]:
# Model Architecture
input_layer = keras.layers.Input(shape=(4,))
dense_1 = keras.layers.Dense(64, activation='relu')(input_layer)
logits = keras.layers.Dense(3)(dense_1) # Here 3 represents number of classes
model = keras.Model(inputs=input_layer, outputs=logits)

In [0]:
# Compile model
custom_loss = keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',
              loss=custom_loss,
              metrics=['accuracy']
             )

In [5]:
# Fit model - without label smoothing
history = model.fit(X, keras.utils.to_categorical(Y),
                    epochs=150,
                    batch_size=8)

Epoch 1/150
19/19 [==============================] - 0s 1ms/step - loss: 1.0709 - accuracy: 0.4400
Epoch 2/150
19/19 [==============================] - 0s 1ms/step - loss: 0.8556 - accuracy: 0.6000
Epoch 3/150
19/19 [==============================] - 0s 1ms/step - loss: 0.7170 - accuracy: 0.8000
Epoch 4/150
19/19 [==============================] - 0s 1ms/step - loss: 0.6329 - accuracy: 0.8467
Epoch 5/150
19/19 [==============================] - 0s 1ms/step - loss: 0.5750 - accuracy: 0.9533
Epoch 6/150
19/19 [==============================] - 0s 1ms/step - loss: 0.5355 - accuracy: 0.7267
Epoch 7/150
19/19 [==============================] - 0s 1ms/step - loss: 0.4919 - accuracy: 0.9600
Epoch 8/150
19/19 [==============================] - 0s 1ms/step - loss: 0.4684 - accuracy: 0.9000
Epoch 9/150
19/19 [==============================] - 0s 1ms/step - loss: 0.4421 - accuracy: 0.9400
Epoch 10/150
19/19 [==============================] - 0s 1ms/step - loss: 0.4208 - accuracy: 0.9733
Epoch 11/

In [0]:
# Logit prediction
y_pred = model.predict(X)

In [12]:
# ECE result before calibration
num_bins = 50
labels_true = tf.convert_to_tensor(Y, dtype=tf.int32, name='labels_true')
logits = tf.convert_to_tensor(y_pred, dtype=tf.float32, name='logits')

tfp.stats.expected_calibration_error(num_bins=num_bins, 
                                     logits=logits, 
                                     labels_true=labels_true)

<tf.Tensor: shape=(), dtype=float32, numpy=0.05008378>

In [9]:
# Temperature Scaling

temp = tf.Variable(initial_value=1.0, trainable=True, dtype=tf.float32) 

def compute_loss():
    y_pred_model_w_temp = tf.math.divide(y_pred, temp)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(\
                                tf.convert_to_tensor(keras.utils.to_categorical(Y)), y_pred_model_w_temp))
    return loss

optimizer = tf.optimizers.Adam(learning_rate=0.01)

print('Temperature Initial value: {}'.format(temp.numpy()))

for i in range(300):
    opts = optimizer.minimize(compute_loss, var_list=[temp])


print('Temperature Final value: {}'.format(temp.numpy()))

Temperature Initial value: 1.0
Temperature Final value: 0.5670669078826904


In [11]:
# ECE result after calibration
y_pred_model_w_temp = tf.math.divide(y_pred, temp)
num_bins = 50
labels_true = tf.convert_to_tensor(Y, dtype=tf.int32, name='labels_true')
logits = tf.convert_to_tensor(y_pred_model_w_temp, dtype=tf.float32, name='logits')

tfp.stats.expected_calibration_error(num_bins=num_bins, 
                                     logits=logits, 
                                     labels_true=labels_true)

<tf.Tensor: shape=(), dtype=float32, numpy=0.031124191>